In [6]:
# from btree import BTree, Node

In [7]:
class Node:
    def __init__(self):
        self.keys = []
        self.values = []
        self.children = []

    def contains_key(self, key):
        return key in self.keys

    def is_leaf(self):
        return len(self.children) == 0

    def get_insert_index(self, key):
        for i, k in enumerate(self.keys):
            if isinstance(key, str) and isinstance(k, str):
                if key < k:
                    return i
            elif isinstance(key, int) and isinstance(k, int):
                if key < k:
                    return i
            elif isinstance(key, str) and isinstance(k, int):
                return i
            elif isinstance(key, int) and isinstance(k, str):
                continue
        return len(self.keys)

    def insert_entry(self, index, key, value):
        self.keys.insert(index, key)
        self.values.insert(index, value)

    def split(self):
        mid = len(self.keys) // 2
        parent = Node()
        parent.keys = self.keys[mid:]
        parent.values = self.values[mid:]
        parent.children = self.children[mid:]
        self.keys = self.keys[:mid]
        self.values = self.values[:mid]
        self.children = self.children[:mid]
        return parent


# Rest of the code remains the same


class BTree:
    def __init__(self, split_threshold):
        self.root = Node()
        self.split_threshold = split_threshold
        self.height = 1
        self.size = 0

    def __len__(self):
        return self.size

    def _find_node(self, current_node, key):
        if current_node.contains_key(key):
            return current_node
        if current_node.is_leaf():
            return None
        child_index = current_node.get_insert_index(key)
        return self._find_node(current_node.children[child_index], key)

    def contains(self, key):
        node = self._find_node(self.root, key)
        return node is not None

    def _add(self, current_node, key, value):
        if current_node.is_leaf():
            index = 0
            while index < len(current_node.keys):
                if type(key) == type(current_node.keys[index]):
                    if key >= current_node.keys[index]:
                        index += 1
                    else:
                        break
                else:
                    if isinstance(key, str) and isinstance(current_node.keys[index], int):
                        index += 1
                    else:
                        break
            current_node.insert_entry(index, key, value)
        else:
            child_index = current_node.get_insert_index(key)
            self._add(current_node.children[child_index], key, value)
        if len(current_node.keys) > self.split_threshold:
            parent = current_node.split()
            if current_node == self.root:
                new_root = Node()
                new_root.keys = [current_node.keys.pop()]
                new_root.values = [current_node.values.pop()]
                new_root.children = [current_node, parent]
                self.root = new_root
                self.height += 1
    def add(self, key, value):
        if not self.contains(key):
            self._add(self.root, key, value)
            self.size += 1

    def get_value(self, key):
        node = self._find_node(self.root, key)
        if node is None:
            return None
        index = node.keys.index(key)
        return node.values[index]



In [18]:
class KVStore(BTree):    
    
#     Initializing
    def __init__(self):
        super().__init__(2)    # extends the BTree class, and set split_threshold as 2
        
    def add(self, key, value):
        if not self.contains(key):
            super().add(key, value)
            
    def __getitem__(self, key):
        return self.get_value(key)
        
    def __setitem__(self, key, value):
        self.add(key, value)
    
    def __contains__(self, key):
        return self.contains(key)

    def _range_query(self, range_start, range_end, current_node, min_key, max_key):
        if min_key is None:
            min_key = float('-inf')
        if max_key is None:
            max_key = float('inf')
        
        if range_start > max_key or range_end < min_key:
            return []
        
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key and key <= range_end:
                results.append(current_node.values[i])
        
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node.keys) else max_key
                results += self._range_query(range_start, range_end, child, new_min_key, new_max_key)
                
            # Check if the range extends beyond the last key in this node
            if range_end > current_node.keys[-1]:
                results += self._range_query(range_start, range_end, current_node.children[-1], current_node.keys[-1], max_key)
        
        return resultsssss

    def range_query(self, range_start, range_end):
        return self._range_query(range_start, range_end, self.root, None, None)

In [19]:
# # add assertions that ensure that the state of the object is what it should be
# kv = KVStore()
# assert kv.split_threshold == 2, "The split is not equal to 2."
# kv.add(1, 'apple')
# assert kv.get_value(1)=='apple'

In [20]:
# create instance
kv = KVStore()
# add in key-value entries
kv[1]='banana'
kv['second'] = 2
kv[2]='Andy'
kv[3]='Fedez'
kv['second'] = 2

assert kv[1]=='banana'   # testing getter & setter
assert 'second' in kv    # testing In Operator